# Notes

This assignment is devoted to `pandas`. It covers indexing and filtering, and some `groupby` and `join` operations. The assignment roughly corresponds to Week 4 and the beginning of Week 5 of the course.

The main dataset you'll be using is [Titanic](https://www.kaggle.com/c/titanic).

In [1]:
%pylab inline
plt.style.use("bmh")

Populating the interactive namespace from numpy and matplotlib


In [2]:
plt.rcParams["figure.figsize"] = (6,6)

In [3]:
import numpy as np
import pandas as pd

In [4]:
STUDENT = "Avshalom  Manevich"
ASSIGNMENT = 4
TEST = False

In [5]:
if TEST:
    import solutions
    total_grade = 0
    MAX_POINTS = 16

# Indexing and filtering

### 1. Fixing age (1 point).

There are several known mistakes in the Titanic dataset.

Namely, [Julia Florence Siegel](https://www.encyclopedia-titanica.org/titanic-survivor/julia-florence-cavendish.html) (Mrs. Tyrell William Cavendish) is mistakenly marked as being 76 years old (the age she actually died, but many years after Titanic).

You must replace age value for her with her actual age at the time (25) and return the dataset. Input is indexed with `PassengerId` and is a concatenation of train and test sets. You must return a copy of the dataframe, and not perform replacement in original dataframe. Structure and indexing must be the same as in input.

In [8]:
def fix_age(df):
    df_c = df.copy(deep = True)
    idx, = df_c[df_c['Name'].str.contains("Julia Florence Siegel")].index
    df_c.loc[idx, "Age"] = 25
    return df_c

titanic_train = pd.read_csv("titanic/train.csv", index_col="PassengerId")
titanic_test = pd.read_csv("titanic/test.csv", index_col="PassengerId")
titanic = pd.concat([titanic_train, titanic_test], sort=False)
# df_c = fix_age(titanic)
titanic[titanic['Name'].str.contains("Julia Florence Siegel")]

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
988,NaN,1,"Cavendish, Mrs. Tyrell William (Julia Florence...",female,76.0,1,0,19877,78.85,C46,S


In [9]:
PROBLEM_ID = 1

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, fix_age)

### 2. Embarkment port distribution (1 point).

You must find the value counts for embarkment port (`Embarked` column) for the passengers, who travelled in 3-d class, were male and between 20 and 30 years old (both inclusive). No need to treat missing values separately.

Input is indexed with `PassengerId` and is a concatenation of train and test sets. You must return series, indexed with values from `Embarked`, according to `.value_counts()` method semantics.

In [13]:
def embarked_stats(df):
    return df[(df['Pclass'] == 3) & (df['Sex'] == 'male') & (df['Age'] >= 20) & (df['Age'] <= 30)].Embarked.value_counts()
# embarked_stats(titanic)
embarked_stats(titanic)

S    132
C     21
Q      7
Name: Embarked, dtype: int64

In [14]:
PROBLEM_ID = 2

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, embarked_stats)

### 3. Fill missing age values (1 point).

Some age values are missing in the Titanic dataset. You need to calculate average age, and fill missing age values in `Age` column.

Input is indexed with `PassengerId` and is a concatenation of train and test sets. Output must be a **new** dataframe with the same structure, but without missing values in `Age` column.

In [15]:
def fix_age(df):
    df = df.copy(deep = True)
    mean_age = df.Age.mean()
    df.loc[df.Age.isnull(), 'Age'] = mean_age
    return df

fix_age(titanic)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0.0,3,"Braund, Mr. Owen Harris",male,22.000000,1,0,A/5 21171,7.2500,NaN,S
2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000000,1,0,PC 17599,71.2833,C85,C
3,1.0,3,"Heikkinen, Miss. Laina",female,26.000000,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000000,1,0,113803,53.1000,C123,S
5,0.0,3,"Allen, Mr. William Henry",male,35.000000,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
1305,NaN,3,"Spector, Mr. Woolf",male,29.881138,0,0,A.5. 3236,8.0500,NaN,S
1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.000000,0,0,PC 17758,108.9000,C105,C
1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.500000,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S


In [16]:
PROBLEM_ID = 3

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, fix_age)

### 4. Child travelling alone (1 point).

You must find a child (`Age<10`) on-board, who was travelling without siblings or parents and find a name of her nursemaid.

Input is indexed with `PassengerId` and is a concatenation of train and test sets. Output must be a tuple of two strings, collected from `Name` column, with one being child's name and second being nursemaid's name. It's known, that there's only one child like this.

In [17]:
def get_nursemaid(df):
    child_name_ticket = df.loc[(df['Age'] < 10) & (df['Parch'] == 0), ['Name', 'Ticket']]
    maid_name = df[(df['Ticket'] == child_name_ticket['Ticket'].values[0]) & (df['Age'] > 10)].Name
    return (child_name_ticket['Name'].values[0], maid_name.values[0])

get_nursemaid(titanic)

('Emanuel, Miss. Virginia Ethel', 'Dowdell, Miss. Elizabeth')

In [18]:
PROBLEM_ID = 4

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, get_nursemaid)

### 5. Port with the most children embarked (1 point).

You must find, which port had the largest percentage of children (`Age<10`) embarked, i.e. number of children divided by total number of passengers embarked.

Input is indexed with `PassengerId` and is a concatenation of train and test sets. Output must be a single string with port letter.

In [19]:
def get_port(df):
    counts = df.groupby(['Embarked']).size()
    ratio = df[df['Age'] < 10].groupby(['Embarked']).size() / counts
    return ratio.idxmax()

print(get_port(titanic))

S


In [20]:
PROBLEM_ID = 5

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, get_port)

### 6. Passengers per ticket (2 points).

Calculate average and maximum number of passengers per ticket.

Input is indexed with `PassengerId` and is a concatenation of train and test sets. Output must be a tuple of two values.

In [22]:
def get_ticket_stats(df):
    counts = df['Ticket'].value_counts()
#     print(counts)
    return (counts.mean(), counts.max())

get_ticket_stats(titanic)

(1.4090419806243273, 11)

In [23]:
PROBLEM_ID = 6

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, get_ticket_stats)

### 7. Fare per passenger (3 points).

For each individual ticket, you must calculate fare per person for that ticket, and then calculate averages for each class. Note, that you will need to apply `groupby` and then you may consider using `.first()` of resulting `DataFrameGroupBy`.

Input is indexed with `PassengerId` and is a concatenation of train and test sets. Output must be `Series` with three elements, indexed by class.

In [24]:
def get_fare_per_pass(df):
#     print(df[df['Ticket'] == '110152']['Fare'])
    df = df[['Ticket','Pclass', 'Fare']].groupby(['Pclass','Ticket']).apply(lambda x: x['Fare'] / len(x.index) )
    df = df.to_frame()
    df = df.groupby('Pclass').mean().squeeze()
    return df

# get_fare_per_pass(titanic)

In [25]:
PROBLEM_ID = 7

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, get_fare_per_pass)

### 8. Fill missing age values (3 points).

In problem 3 you filled missing age values with average for all passengers. Now, you need to fill them according to class and sex. For example, for a female passenger from 2d class, missing age value must be filled with average age of females in 2d class.

In this problem, you may need joins and `.apply()`, although there are several ways to get the same result.

Input is indexed with `PassengerId` and is a concatenation of train and test sets. Output must be a **new** dataframe with the same structure as input, but without missing values in `Age` column.

In [26]:
def fix_age_groupped(df):
    df = df.copy()
    grouped = df.groupby(['Pclass', 'Sex'])
    means = grouped['Age'].mean().to_frame()
    for index, row in means.iterrows():
        c, sex = index
        age = row.values[0]
        df.loc[(df['Pclass'] == c) & (df['Sex'] == sex) & (df['Age'].isnull()), 'Age'] = age
    return df
# fix_age_groupped(titanic)

In [27]:
PROBLEM_ID = 8

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, fix_age_groupped)

### 9. Finding couples (3 points).

Based on the code from Lecture 5, build a dataframe of couples. Filter it by survival status: select those couples, in which only one of spouses survived or none of two. Built survival statistics by class, i.e. ratio of couples with partial survival divided by total number of couples in class.

Input is indexed with `PassengerId` and is a concatenation of train and test sets. Output must be `Series` with three elements indexed by values from `Pclass` column.

In [74]:
def find_couples(df):
    df = df.copy()
    df = df[(df['Survived'].notnull())]
    df = df[(df['SibSp'] == 1) & (df['Parch'] == 0)].sort_values(by=['Ticket'])
    df = df[((df['Sex'] == 'female') & (df['Name'].str.contains('Mrs.'))) | (df['Sex'] == 'male')]
    df = df.replace(re.compile(r'\s+\(.*\)'), '').replace(re.compile("Mrs."), "Mr.")
    df = df.groupby(['Ticket']).filter(lambda x: len(x) == 2)
    couples_per_class = df.groupby(['Pclass']).size()
    df = df.groupby(['Ticket']).filter(lambda x: ((len(x) == 2) and (sum(x['Survived']) != 2)))
    partial_survivers = df.groupby(['Pclass']).size()
    return partial_survivers / couples_per_class

find_couples(titanic)

Pclass
1    0.500000
2    0.833333
3    1.000000
dtype: float64

In [29]:
PROBLEM_ID = 8

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, find_couples)